In [5]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import eli5 
from collections import Counter
import gc

In [6]:
#Get Data
df = pd.read_hdf(r"C:\Users\Jakub\Desktop\python\data.h5")

In [8]:
df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)


df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()


194

In [9]:
def feature_engineering(df):
    df_customers = (
        df
        .groupby("customer_id")
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        ).reset_index()
    )
    
    
    
    return df_customers

In [10]:
def get_feats(df_customers, black_list=["most_revenue_customer"]):
    feats = list(df_customers.select_dtypes([np.number, bool]).columns)
    return [x for x in feats if x not in black_list]

In [11]:
def get_X_y(df_customers, top_customers, feats):
    df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)
    
    X = df_customers[feats].values
    y = df_customers["most_revenue_customer"].values
    
    return X, y

In [12]:
def train_and_get_scores(model, X, y, scoring="accuracy", cv=5):

    scores = cross_val_score(model, X, y, scoring=scoring, cv=cv)
    return np.mean(scores), np.std(scores)

In [13]:
# 1st model

df_customers = feature_engineering(df)
feats = get_feats(df_customers)
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=5)

train_and_get_scores(model, X, y)

(0.9996598639455783, 0.0004165798882284457)

In [14]:
# 2 nd Model. Exclude Sum Price total
df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=5)

train_and_get_scores(model, X, y)

(0.772922275293096, 0.2656789279474872)

In [15]:
# 3rd model using xgboost
df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.3)

train_and_get_scores(model, X, y)

(0.8106769431176726, 0.08696513375768444)

In [16]:
#Check feature importance

model.fit(X, y)

eli5.show_weights(model, feature_names=feats)

Weight,Feature
0.6835,sum_quantity
0.1309,count_orders
0.0776,sum_price_unit
0.0545,count_unq_products
0.0298,prob_canceled
0.0235,customer_id
0,count_unq_countries


In [17]:
def feature_engineering(df):
    
    def counter(vals):
        cntr = Counter()
        cntr.update(vals)
        return cntr
    
    df_customers = (
        df
        .groupby("customer_id")
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            count_by_products=("product_id", lambda x:  counter(x) ),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        ).reset_index()
    )
    
    
    return df_customers

In [18]:
df_customers  = feature_engineering(df)
df_customers.head()

,customer_id,count_orders,count_unq_products,count_by_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled
0,0,159,93,"{0: 89, 1: 73, 2: 69, 3: 68, 4: 32, 5: 36, 6: ...",22976,895827,5746671,1,0.010966
1,1,40,184,"{8: 8, 9: 4, 10: 4, 11: 4, 12: 3, 13: 15, 14: ...",3514,205962,910113,1,0.097727
2,2,30,139,"{25: 9, 26: 7, 27: 7, 28: 3, 29: 4, 30: 10, 31...",8971,95500,1337464,1,0.009050
3,3,9,40,"{44: 7, 1781: 1, 1179: 1, 112: 1, 508: 3, 581:...",1108,18626,226381,1,0.000000
4,4,14,11,"{54: 14, 1496: 2, 2288: 1, 2315: 1, 3432: 1, 4...",390,20028,346978,1,0.307692


In [21]:
df_customers["count_by_products"]

df_count_products = df_customers["count_by_products"].apply(pd.Series).fillna(-1)
df_count_products.columns = ["product_{}".format(x) for x in df_count_products.columns]

df_count_products.head(5)

# -1 = no order

,product_0,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_45,product_46,...,product_3834,product_3845,product_3847,product_3849,product_3852,product_3127,product_4082,product_4148,product_4201,product_3952
0,89.0,73.0,69.0,68.0,32.0,36.0,34.0,35.0,67.0,49.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,6.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
#Concatanate fields `df_count_products` & `df_customers` 
df_customers = pd.concat([df_customers, df_count_products], axis=1)
df_customers.shape

(5879, 3887)

In [23]:
#Train
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.3)

train_and_get_scores(model, X, y)

(0.8479219858156029, 0.09102615843251025)

In [24]:
model.fit(X, y)
eli5.show_weights(model, feature_names=feats, top=50)
#Prod_545
#Prod_99


Weight,Feature
0.0992,sum_quantity
0.0179,product_545
0.0168,product_99
0.0145,count_orders
0.0132,product_1777
0.0131,product_847
0.0124,product_1280
0.0120,product_209
0.0103,product_1604
0.0101,product_2908


In [27]:
#CNT Clients Orders
customer_ids_by_product = set(df[ df["product_id"] == 545 ]["customer_id"].unique())
len(customer_ids_by_product )

168

In [26]:
df_customers[ df_customers.customer_id.isin(customer_ids_by_product) ]["most_revenue_customer"].mean()

0.4166666666666667

In [31]:
df

,order_id,customer_id,product_id,quantity,price_unit,price_total,country_id,order_date,is_canceled
0,0,0,0,6,254,1524,0,2010-12-01 08:26:00,False
1,0,0,1,6,339,2034,0,2010-12-01 08:26:00,False
2,0,0,2,8,275,2200,0,2010-12-01 08:26:00,False
3,0,0,3,6,339,2034,0,2010-12-01 08:26:00,False
4,0,0,3,6,339,2034,0,2010-12-01 08:26:00,False
...,...,...,...,...,...,...,...,...,...
820901,1070,620,362,2,295,590,0,2010-12-09 20:01:00,False
820902,1070,620,395,1,375,375,0,2010-12-09 20:01:00,False
820903,1070,620,364,1,375,375,0,2010-12-09 20:01:00,False
820904,1070,620,1242,2,375,750,0,2010-12-09 20:01:00,False


In [30]:
df[ df.product_id == 545 ]["price_unit"].value_counts()

125    126
39     103
42       5
246      3
106      1
Name: price_unit, dtype: int64